In [45]:
import requests
import urllib.parse
import time

In [23]:
import pandas as pd
from pymongo import MongoClient
import time
from bs4 import BeautifulSoup

mc = MongoClient()
db  = mc['raw_restaurants']
biz = db['restaurants']
users = db['users']
rv = db['reviews']

In [97]:
df_user = pd.DataFrame(list(users.find()))

In [102]:
df_user.shape

(11633, 6)

In [98]:
df_user.head().T

,0,1,2,3,4
_id,5bf9bd137988471b045bc144,5bf9bd137988471b045bc146,5bf9bd137988471b045bc148,5bf9bd137988471b045bc14a,5bf9bd137988471b045bc14c
id,oxo1EWFAR8nQJoSFvZG6xA,DP4BchE8S9BMxsh3lcBTrQ,53081ApXllnHtkNipUCnDg,_3KTfz0hcPl_yq4jOyOPiA,QtnPqgZsLPrlt3hzvWowzg
image_url,https://s3-media4.fl.yelpcdn.com/photo/RHGmUNE...,https://s3-media2.fl.yelpcdn.com/photo/Qk0CFiy...,https://s3-media3.fl.yelpcdn.com/photo/uOrz3QQ...,https://s3-media1.fl.yelpcdn.com/photo/L_4qfL8...,https://s3-media1.fl.yelpcdn.com/photo/7xN-KWQ...
name,jessica c.,Stacy S.,Nicole V.,Tracy L.,Deven B.
page,b'<!DOCTYPE HTML>\n\n<!--[if lt IE 7 ]> <html ...,b'<!DOCTYPE HTML>\n\n<!--[if lt IE 7 ]> <html ...,b'<!DOCTYPE HTML>\n\n<!--[if lt IE 7 ]> <html ...,b'<!DOCTYPE HTML>\n\n<!--[if lt IE 7 ]> <html ...,b'<!DOCTYPE HTML>\n\n<!--[if lt IE 7 ]> <html ...
profile_url,https://www.yelp.com/user_details?userid=oxo1E...,https://www.yelp.com/user_details?userid=DP4Bc...,https://www.yelp.com/user_details?userid=53081...,https://www.yelp.com/user_details?userid=_3KTf...,https://www.yelp.com/user_details?userid=QtnPq...


In [99]:
df_rv = pd.DataFrame(list(rv.find()))

In [29]:
df_rv.columns

Index(['_id', 'id', 'rating', 'text', 'time_created', 'url', 'user'], dtype='object')

In [32]:
df_rv['user_id'] = df_rv['user'].map(lambda x : x['id'])

In [101]:
len(df_rv)

18493

In [40]:
df_user.shape, df_user.columns

((6271, 5),
 Index(['_id', 'id', 'image_url', 'name', 'profile_url'], dtype='object'))

In [36]:
mini_users = df_user.profile_url[:7]

In [53]:
def scrape_mongo(url):
    """Return the HTML content of url."""
## May be check 1st and 2nd rows I will outside of this function
    cached_page = users.find_one({'profile_url': url}).get('page')
    if cached_page is None:
        print(f"Fetching {url}")
        response = requests.get(url)
        if response.status_code == 200:
            users.update_one({'profile_url': url},
                              {'$set': {'page': response.content}})   
        else:
            users.update_one({'profile_url': url},
                              {'$set': {'page': 'Error'}})   
    return users.find_one({'profile_url': url}).get('page')
    

In [54]:
mini_scrape = []
for url in mini_users:
    mini_scrape.append(scrape_mongo(url))
    time.sleep(33)
    

Fetching https://www.yelp.com/user_details?userid=DP4BchE8S9BMxsh3lcBTrQ
Fetching https://www.yelp.com/user_details?userid=53081ApXllnHtkNipUCnDg
Fetching https://www.yelp.com/user_details?userid=_3KTfz0hcPl_yq4jOyOPiA
Fetching https://www.yelp.com/user_details?userid=QtnPqgZsLPrlt3hzvWowzg
Fetching https://www.yelp.com/user_details?userid=rqlngRO2Hj0sX9WpEQ_vbg
Fetching https://www.yelp.com/user_details?userid=7Zk12TsWz5CJgRERnICPRw


In [58]:
len(mini_users), len(mini_scrape)

(7, 7)

In [103]:
mini_users[1]

'https://www.yelp.com/user_details?userid=DP4BchE8S9BMxsh3lcBTrQ'

In [123]:
mini_users[1].split('?')[-1].split('=')[-1]

'DP4BchE8S9BMxsh3lcBTrQ'

In [141]:
url2 ='https://www.yelp.com/user_details_reviews_self?rec_pagestart=10&userid=DP4BchE8S9BMxsh3lcBTrQ'

In [142]:
url2.split('=')


['https://www.yelp.com/user_details_reviews_self?rec_pagestart',
 '10&userid',
 'DP4BchE8S9BMxsh3lcBTrQ']

In [66]:
stacy =  BeautifulSoup(mini_scrape[1], 'html.parser')

<i> Amount of reivews </i>

In [76]:
st_rev =stacy.select_one('li.review-count').select_one('strong').text
int(st_rev)

27

In [92]:
revs = stacy.select('div.review')
type(revs), len(revs), type(revs[0])

(list, 10, bs4.element.Tag)

In [96]:
bjs = revs[0]
bjs.contents

['\n', <div class="review-topbar">
 <div class="media-block media-block--12 biz-listing-medium">
 <div class="media-avatar">
 <div class="photo-box pb-60s">
 <a class="js-analytics-click" data-analytics-label="biz-photo" href="/biz/bjs-restaurant-and-brewhouse-tukwila">
 <img alt="BJ's Restaurant &amp; Brewhouse" class="photo-box-img" height="60" src="https://s3-media3.fl.yelpcdn.com/bphoto/tK-c_jWq5TnSEuPVN07MNg/60s.jpg" srcset="https://s3-media3.fl.yelpcdn.com/bphoto/tK-c_jWq5TnSEuPVN07MNg/90s.jpg 1.50x,https://s3-media3.fl.yelpcdn.com/bphoto/tK-c_jWq5TnSEuPVN07MNg/168s.jpg 2.80x,https://s3-media3.fl.yelpcdn.com/bphoto/tK-c_jWq5TnSEuPVN07MNg/ms.jpg 1.67x,https://s3-media3.fl.yelpcdn.com/bphoto/tK-c_jWq5TnSEuPVN07MNg/180s.jpg 3.00x,https://s3-media3.fl.yelpcdn.com/bphoto/tK-c_jWq5TnSEuPVN07MNg/120s.jpg 2.00x" width="60"/>
 </a>
 </div>
 </div>
 <div class="media-story">
 <div class="media-title clearfix">
 <a class="biz-name js-analytics-click" data-analytics-label="biz-name" data-hov

In [126]:
cat = bjs.select_one('span.rating-qualifier')
cat.text.strip()

'11/6/2018'

In [111]:
bjs.select_one('div.i-stars').get_attribute_list('title')[0].split()[0]

'1.0'

In [138]:
bizname = bjs.select_one('a.biz-name')
bizname.attrs['href'].split('/')[-1]

'bjs-restaurant-and-brewhouse-tukwila'

In [149]:
%%time
time.sleep(1)

CPU times: user 765 µs, sys: 1.35 ms, total: 2.11 ms
Wall time: 1 s


In [153]:
from add_by_scrap import parse_review, scrap_by_users

In [156]:
%%time
res3 = scrap_by_users(mini_users[3])

CPU times: user 10.6 s, sys: 170 ms, total: 10.7 s
Wall time: 25 s


In [157]:
rv_s = db['reviews_scrap']


In [159]:
from starting_db import add_to_database_if_new

In [160]:
%%time
for x in res3:
    add_to_database_if_new(x,rv_s)

CPU times: user 41.6 ms, sys: 4.95 ms, total: 46.5 ms
Wall time: 194 ms


In [176]:
df_rv_sc = pd.DataFrame(list(rv_s.find()))

In [177]:
df_rv_sc

,_id,alias,category,date,id,rating,text,user_id
0,5bfb5ed579884720729dd209,srivilai-thai-cuisine-seattle,\nThai\n,10/13/2018,ErsYo_9v8--feFCy92tymQ,5.0,Fantastic Thai food with warm service. Authent...,_3KTfz0hcPl_yq4jOyOPiA
1,5bfb5ed579884720729dd20a,ohs-sandwiches-seattle-2,"\nSandwiches,\n Bubble Tea\n",10/10/2018,W1NnSItt-SwWHrzkRtZ44g,5.0,Fantastic Vietnamese sandwich: great flavors a...,_3KTfz0hcPl_yq4jOyOPiA
2,5bfb5ed579884720729dd20b,saison-san-francisco-2,\nAmerican (New)\n,9/21/2018,3Q5uzROHYl1i2B_t2sIl7w,3.0,A fine dining restaurant that is in the midst ...,_3KTfz0hcPl_yq4jOyOPiA
3,5bfb5ed579884720729dd20c,international-smoke-san-francisco-2,"\nBarbeque,\n Smokehouse,\n...",12/23/2017,-GYe-Aq7NcNJ3u-U6fmwMg,5.0,Great restaurant with an interesting mix of in...,_3KTfz0hcPl_yq4jOyOPiA
4,5bfb5ed579884720729dd20d,supercuts-san-francisco,\nHair Salons\n,12/23/2017,pPlsW81ntC2qbF0QOe6pLQ,5.0,We always have a good experience here with Ani...,_3KTfz0hcPl_yq4jOyOPiA
5,5bfb5ed579884720729dd20e,schuberts-bakery-san-francisco,"\nBakeries,\n Desserts\n",11/22/2017,qgpe9AFPEQ7nc5NAnuACeA,2.0,Service at this bakery has gone to the toilet....,_3KTfz0hcPl_yq4jOyOPiA
6,5bfb5ed579884720729dd20f,dim-n-den-sum-koloa-3,"\nFood Trucks,\n Asian Fusi...",9/7/2017,XwZeVkIJ4fsZQ4X-UorzhQ,5.0,Great crab sandwich and very tasty fries. Crun...,_3KTfz0hcPl_yq4jOyOPiA
7,5bfb5ed579884720729dd210,uncles-shave-ice-koloa,"\nShaved Ice,\n Desserts\n",9/6/2017,94IFjTfoJhe2v3hJDyzpaA,1.0,"Rude employees, expensive, and not a lot of fl...",_3KTfz0hcPl_yq4jOyOPiA
8,5bfb5ed579884720729dd211,hi-joyful-singapore,\nSeafood\n,4/12/2017,MqiIrbup900dQmA5lizqJw,3.0,We got Singapore noodles and beer. Pretty good...,_3KTfz0hcPl_yq4jOyOPiA
9,5bfb5ed579884720729dd212,thai-nghiep-ky-mi-gia-san-francisco,"\nVietnamese,\n Chinese,\n ...",3/8/2017,Ctzuk8Ro1XcQsFtNLeEhkw,5.0,Their braised duck with egg noodles (#10 in th...,_3KTfz0hcPl_yq4jOyOPiA


In [169]:
users.find_one()

{'_id': ObjectId('5bf9bd137988471b045bc144'),
 'id': 'oxo1EWFAR8nQJoSFvZG6xA',
 'profile_url': 'https://www.yelp.com/user_details?userid=oxo1EWFAR8nQJoSFvZG6xA',
 'image_url': 'https://s3-media4.fl.yelpcdn.com/photo/RHGmUNE3MviXEJDY4okWCQ/o.jpg',
 'name': 'jessica c.',
 'page': b'<!DOCTYPE HTML>\n\n<!--[if lt IE 7 ]> <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie6 ie ltie9 ltie8 no-js" lang="en"> <![endif]-->\n<!--[if IE 7 ]>    <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie7 ie ltie9 ltie8 no-js" lang="en"> <![endif]-->\n<!--[if IE 8 ]>    <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie8 ie ltie9 no-js" lang="en"> <![endif]-->\n<!--[if IE 9 ]>    <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie9 ie no-js" lang="en"> <![endif]-->\n<!--[if (gt IE 9)|!(IE)]><!--> <html xmlns:fb="http://www.facebook.com/2008/fbml" class="no-js" lang="en"> <!--<![endif]-->\n    <head>\n        <script>\n            (function() {\n                var main

In [173]:
mini_u = pd.DataFrame(list(users.find(limit = 10)))
mini_u.shape

(10, 6)

In [174]:
mini_u

,_id,id,image_url,name,page,profile_url
0,5bf9bd137988471b045bc144,oxo1EWFAR8nQJoSFvZG6xA,https://s3-media4.fl.yelpcdn.com/photo/RHGmUNE...,jessica c.,b'<!DOCTYPE HTML>\n\n<!--[if lt IE 7 ]> <html ...,https://www.yelp.com/user_details?userid=oxo1E...
1,5bf9bd137988471b045bc146,DP4BchE8S9BMxsh3lcBTrQ,https://s3-media2.fl.yelpcdn.com/photo/Qk0CFiy...,Stacy S.,b'<!DOCTYPE HTML>\n\n<!--[if lt IE 7 ]> <html ...,https://www.yelp.com/user_details?userid=DP4Bc...
2,5bf9bd137988471b045bc148,53081ApXllnHtkNipUCnDg,https://s3-media3.fl.yelpcdn.com/photo/uOrz3QQ...,Nicole V.,b'<!DOCTYPE HTML>\n\n<!--[if lt IE 7 ]> <html ...,https://www.yelp.com/user_details?userid=53081...
3,5bf9bd137988471b045bc14a,_3KTfz0hcPl_yq4jOyOPiA,https://s3-media1.fl.yelpcdn.com/photo/L_4qfL8...,Tracy L.,b'<!DOCTYPE HTML>\n\n<!--[if lt IE 7 ]> <html ...,https://www.yelp.com/user_details?userid=_3KTf...
4,5bf9bd137988471b045bc14c,QtnPqgZsLPrlt3hzvWowzg,https://s3-media1.fl.yelpcdn.com/photo/7xN-KWQ...,Deven B.,b'<!DOCTYPE HTML>\n\n<!--[if lt IE 7 ]> <html ...,https://www.yelp.com/user_details?userid=QtnPq...
5,5bf9bd137988471b045bc14e,rqlngRO2Hj0sX9WpEQ_vbg,https://s3-media1.fl.yelpcdn.com/photo/owk8-fz...,Nichole B.,b'<!DOCTYPE HTML>\n\n<!--[if lt IE 7 ]> <html ...,https://www.yelp.com/user_details?userid=rqlng...
6,5bf9bd147988471b045bc150,7Zk12TsWz5CJgRERnICPRw,https://s3-media1.fl.yelpcdn.com/photo/tR2gI3A...,Ben C.,b'<!DOCTYPE HTML>\n\n<!--[if lt IE 7 ]> <html ...,https://www.yelp.com/user_details?userid=7Zk12...
7,5bf9bd147988471b045bc152,RNbgsPuTi6UKh2D-W9xIig,https://s3-media4.fl.yelpcdn.com/photo/S3Wnibn...,Bryana S.,NaN,https://www.yelp.com/user_details?userid=RNbgs...
8,5bf9bd147988471b045bc154,q2KEuDWUM8_ErdhZakos8A,https://s3-media2.fl.yelpcdn.com/photo/yGWrlAR...,Joline R.,NaN,https://www.yelp.com/user_details?userid=q2KEu...
9,5bf9bd147988471b045bc156,KYoKrveCUg4CC9YcSvkApQ,https://s3-media4.fl.yelpcdn.com/photo/7BRLalm...,Audrey C.,NaN,https://www.yelp.com/user_details?userid=KYoKr...


## Start scraping .

Start with 100 users 

In [ ]:
from add_by_scrap import build_review_by_scrap